In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import requests
from bs4 import BeautifulSoup

## 1. Obtain the 200 top-ranking universities in www.topuniversities.com (ranking 2018). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the details page. Store the resulting dataset in a pandas DataFrame and answer the following questions:

Using the postman plugin on google chrome, we found the json representing the list of the universities:

https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508252855868

We want to scrap those data from the site:
* name 
* rank
* country
* region 
* number of faculty members (international and total)
* number of students (international and total)

In the JSON we retrieved, there is already the **name**, **rank**, **country** and **region**. 

In [2]:
#json to dataframe using panda
json_data = json.load(open('TopUniRanking.json'))
df = json_normalize(json_data['data'])
#the university are sorted by ranks, so we just need the 200 first indexes
dfTopUni200 = df.head(200)
#drop useless column information
dfTopUni200 = dfTopUni200.drop(["cc","core_id","logo","guide"],axis=1)
dfTopUni200.head(10)

,country,nid,rank_display,region,score,stars,title,url
0,United States,294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,United States,297282,2,North America,98.7,5,Stanford University,/universities/stanford-university
2,United States,294270,3,North America,98.4,5,Harvard University,/universities/harvard-university
3,United States,294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...
4,United Kingdom,294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge
5,United Kingdom,294654,6,Europe,95.3,5,University of Oxford,/universities/university-oxford
6,United Kingdom,294014,7,Europe,94.6,None,UCL (University College London),/universities/ucl-university-college-london
7,United Kingdom,294030,8,Europe,93.7,None,Imperial College London,/universities/imperial-college-london
8,United States,294536,9,North America,93.5,5,University of Chicago,/universities/university-chicago
9,Switzerland,294432,10,Europe,93.3,None,ETH Zurich - Swiss Federal Institute of Techno...,/universities/eth-zurich-swiss-federal-institu...


We now have the general data for the 200 first universities. Then we need to scrap data on each unique page of each university to retrieve those 4 informations: 
* **number of total faculty members** 
* **number of international faculty members**
* **number of total students** 
* **number of international students**

The url of each university is contained in our previous dataFrame. And the link to the pages are written this way:
 **https://www.topuniversities.com+url** 
 
Using developper tool in google chrome, we found that the 4 datas we want are located inside unique html objects that have as class: **total faculty**, **inter faculty**, **total student** and **total inter**.
So to find those datas we filter by class name the html page, then we retreive the specific associated information.

### **Warning :** the next cell requiers internet to work as intended and it lasts around 2 minutes of computation.


In [5]:
listUrl = dfTopUni200['url']
#create a temporary dataframe
index = range(200)
columns = ["nbr_faculty_members","nbr_international_faculty_members","nbr_total_students","nbr_international_total_students"]
temp_df = pd.DataFrame(index=index,columns=columns)

#helper function that filter the html file and return the int corresponding to the special string htmlClass
#return NaN if the object doesn't exist
def findNumberOf(soup,htmlClass):
    filteredHtmlElements = soup.find_all('div', class_=htmlClass)
    #case if a field information is not given
    if(len(filteredHtmlElements)==0):
        return np.nan
    #find the integer value inside the html balise
    t= filteredHtmlElements[0].find('div', class_='number').text
    #clear the input then convert it into an integer
    return int(t.replace('\n', '').replace('\r', '').replace(',', '').replace(' ',''))

#retrieve the 4 informations needed from the url, for each university
for i,url in enumerate(listUrl):
    finalUrl = "https://www.topuniversities.com"+url
    r = requests.get(finalUrl)
    soup = BeautifulSoup(r.text, 'html.parser')
    #todo: scrap with beautifulSoup using the url
    #merge data into the temp dataframe
    temp_df.nbr_faculty_members[i] = findNumberOf(soup,'total faculty')
    temp_df.nbr_international_faculty_members[i] = findNumberOf(soup,'inter faculty')
    temp_df.nbr_total_students[i] = findNumberOf(soup,'total student')
    temp_df.nbr_international_total_students[i] = findNumberOf(soup,'total inter')

Now that we have all the informations we want into this DataFrame, we merge it with the initial one.

In [7]:
#merge the old dataframe with the new information
#fullDataFrameTopUni200 contains all the information of the 200 top universities from www.topuniversities.com
fullDataFrameTopUni200 = dfTopUni200.join(temp_df)
fullDataFrameTopUni200.head(10)

,country,nid,rank_display,region,score,stars,title,url,nbr_faculty_members,nbr_international_faculty_members,nbr_total_students,nbr_international_total_students
0,United States,294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...,2982,1679,11067,3717
1,United States,297282,2,North America,98.7,5,Stanford University,/universities/stanford-university,4285,2042,15878,3611
2,United States,294270,3,North America,98.4,5,Harvard University,/universities/harvard-university,4350,1311,22429,5266
3,United States,294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...,953,350,2255,647
4,United Kingdom,294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge,5490,2278,18770,6699
5,United Kingdom,294654,6,Europe,95.3,5,University of Oxford,/universities/university-oxford,6750,2964,19720,7353
6,United Kingdom,294014,7,Europe,94.6,None,UCL (University College London),/universities/ucl-university-college-london,6345,2554,31080,14854
7,United Kingdom,294030,8,Europe,93.7,None,Imperial College London,/universities/imperial-college-london,3930,2071,16090,8746
8,United States,294536,9,North America,93.5,5,University of Chicago,/universities/university-chicago,2449,635,13557,3379
9,Switzerland,294432,10,Europe,93.3,None,ETH Zurich - Swiss Federal Institute of Techno...,/universities/eth-zurich-swiss-federal-institu...,2477,1886,19815,7563


## Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?

To answer this question we use the previous DataFrame filtering only interesting parameters in column.
Then we can compute the 2 ratios.

Take note that we drop 2 university that doesn't provide enough information on their personnal page.
Indian Institute of Science (IISc) Bangalore doesn't have the number of international faculty member and New York University (NYU) doesn't have any data at all!

In [8]:
#helper function that compute the ratio of faculty member in function of students
def percentFacAndStud (row):
    return row['nbr_faculty_members']/row['nbr_total_students']
#helper function that compute the ratio of international students in function of total students
def percentInterAndStud (row):
    return row['nbr_international_total_students']/row['nbr_total_students']


#hard copy old DF and drop university that doesn't have the infromation required
filteredDataFrame = fullDataFrameTopUni200.copy()[['title','nbr_faculty_members','nbr_international_faculty_members','nbr_total_students','nbr_international_total_students']].dropna()

#create new row with the function percentFacAndStud to compute the ratio faculty member per student
filteredDataFrame['ratio_faculty_per_student'] = filteredDataFrame.apply (lambda row: percentFacAndStud (row),axis=1)
#create new row with the function percentInterAndStud to compute the ratio international students in function of total students
filteredDataFrame['ratio_international_students'] = filteredDataFrame.apply (lambda row: percentInterAndStud (row),axis=1)

#filter the result to answer question (a)
resultDataFrame=filteredDataFrame[['title','ratio_faculty_per_student','nbr_faculty_members','nbr_total_students']]
#return the head of the dataframe, sorted by best ratio faculty per student
resultDataFrame.sort_values(by='ratio_faculty_per_student',ascending=False).head(15)

,title,ratio_faculty_per_student,nbr_faculty_members,nbr_total_students
3,California Institute of Technology (Caltech),0.422616,953,2255
15,Yale University,0.398323,4940,12402
5,University of Oxford,0.342292,6750,19720
4,University of Cambridge,0.292488,5490,18770
16,Johns Hopkins University,0.276353,4462,16146
1,Stanford University,0.269870,4285,15878
0,Massachusetts Institute of Technology (MIT),0.269450,2982,11067
185,University of Rochester,0.266604,2569,9636
18,University of Pennsylvania,0.266437,5499,20639
17,Columbia University,0.247115,6189,25045


The top 3 universities in term of **ratio between faculty members and students** are : 
* California Institute of Technology (Caltech)
* Yale University
* University of Oxford



In [9]:
#filter the result to answer question (b)
resultDataFrame=filteredDataFrame[['title','ratio_international_students','nbr_international_total_students','nbr_total_students']]
#return the head of the dataframe, sorted by best ratio faculty per student
resultDataFrame.sort_values(by='ratio_international_students',ascending=False).head(15)

,title,ratio_international_students,nbr_international_total_students,nbr_total_students
34,London School of Economics and Political Scien...,0.691393,6748,9760
11,Ecole Polytechnique Fédérale de Lausanne (EPFL),0.570047,5896,10343
7,Imperial College London,0.543567,8746,16090
198,Maastricht University,0.502533,8234,16385
47,Carnegie Mellon University,0.478062,6385,13356
6,UCL (University College London),0.477928,14854,31080
91,University of St Andrews,0.457955,4030,8800
41,The University of Melbourne,0.427434,18030,42182
126,Queen Mary University of London,0.421816,6806,16135
25,The University of Hong Kong,0.407144,8230,20214


The top 3 universities in term of **ratio of international students** are:
* London School of Economics and Political Science
* **Ecole Polytechnique Fédérale de Lausanne (EPFL)**	
* Imperial College London

## Answer the previous question aggregating the data by (c) country and (d) region.

In [11]:
#filter the full dataFrame to only releveant parameters
filteredDataFrame = fullDataFrameTopUni200.copy()[['country','region','nbr_faculty_members','nbr_international_faculty_members','nbr_total_students','nbr_international_total_students']]
#aggregate the countries for question (c)
countryDataFrame = filteredDataFrame.drop('region',axis=1).groupby('country').agg('sum')

#the same helper functions are applied to create the 2 ratio into 2 new columns
countryDataFrame['ratio_faculty_per_student'] = countryDataFrame.apply (lambda row: percentFacAndStud (row),axis=1)
countryDataFrame['ratio_international_students'] = countryDataFrame.apply (lambda row: percentInterAndStud (row),axis=1)

countryDataFrame[['ratio_faculty_per_student','nbr_faculty_members','nbr_total_students']].sort_values(by='ratio_faculty_per_student',ascending=False).head(10)

,ratio_faculty_per_student,nbr_faculty_members,nbr_total_students
country,,,
Russia,0.221910,6709,30233
Denmark,0.177261,11916,67223
Saudi Arabia,0.175828,1062,6040
Singapore,0.161530,9444,58466
Malaysia,0.153893,2755,17902
Japan,0.152479,28395,186222
South Korea,0.141721,19851,140071
Switzerland,0.140434,15323,109112
United Kingdom,0.136962,79934,583621


The top 3 countries in term of **ratio between faculty members and students** are:
* Russia
* Denmark
* Saudia Arabia

In [12]:
countryDataFrame[['ratio_international_students','nbr_international_total_students','nbr_total_students']].sort_values(by='ratio_international_students',ascending=False).head(10)

,ratio_international_students,nbr_international_total_students,nbr_total_students
country,,,
Australia,0.352189,106359,301994
United Kingdom,0.341705,199426,583621
Hong Kong,0.310751,24499,78838
Austria,0.309980,19667,63446
Switzerland,0.302396,32995,109112
Singapore,0.276537,16168,58466
Canada,0.260161,73239,281514
New Zealand,0.258215,12439,48173
Ireland,0.235299,8187,34794


The top 3 countries in term of **ratio of international students** are:
* Australia
* United Kingdom	
* Hong Kong

## Answer the previous question for region. (d)

This code is similar to the code of question (c), but instead we aggregate on the label 'region' instead of 'country'.

In [20]:
# same code as for the country, but with region
regionDataFrame = filteredDataFrame.drop('country',axis=1).groupby('region').agg('sum')

#the same helper functions are applied to create the 2 ratio into 2 new columns
regionDataFrame['ratio_faculty_per_student'] = regionDataFrame.apply (lambda row: percentFacAndStud (row),axis=1)
regionDataFrame['ratio_international_students'] = regionDataFrame.apply (lambda row: percentInterAndStud (row),axis=1)
regionDataFrame[['ratio_faculty_per_student','nbr_faculty_members','nbr_total_students']].sort_values(by='ratio_faculty_per_student',ascending=False).head(10)

,ratio_faculty_per_student,nbr_faculty_members,nbr_total_students
region,,,
Asia,0.132260,106734,807003
North America,0.117776,182123,1546353
Europe,0.111564,218358,1957251
Latin America,0.104147,45382,435750
Africa,0.088450,1733,19593
Oceania,0.072385,25347,350167


The top 3 regions in term of **ratio between faculty members and students** are:
* Asia
* North America
* Europe

In [21]:
regionDataFrame[['ratio_international_students','nbr_international_total_students','nbr_total_students']].sort_values(by='ratio_international_students',ascending=False).head(10)

,ratio_international_students,nbr_international_total_students,nbr_total_students
region,,,
Oceania,0.339261,118798,350167
Europe,0.229589,449364,1957251
North America,0.188906,292116,1546353
Africa,0.169703,3325,19593
Asia,0.136431,110100,807003
Latin America,0.084615,36871,435750


The top 3 regions in term of **ratio of international students** are:
* Oceania
* Europe
* North America

# 2. Scrap Site www.timeshighereducation.com

found the json using postman again:

https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json

We want:
    - name 
    - rank
    - country
    - region 
    - number of faculty members (international and total)
    - number of students (international and total)

In [36]:
#json to dataframe using panda
json_data2 = json.load(open('TimesRanking.json'))
df2 = json_normalize(json_data2['data'])
#print(df2.T.index)
yolo123 = df2.head(200)
yolo123

,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...
5,Harvard University,United States,0,Harvard University,466,6,60,private,99.7,8,...,98.4,2,84.2,9,None,"20,326",26%,8.9,"Agriculture & Forestry,Archaeology,Art, Perfor...",/world-university-rankings/harvard-university
6,Princeton University,United States,0,Princeton University,469,7,70,private,99.6,9,...,93.9,6,85.7,7,45 : 55,"7,955",24%,8.3,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/princeton-university
7,Imperial College London,United Kingdom,0,Imperial College London,472,8,80,master_account,96.7,40,...,88.7,12,81.7,13,37 : 63,"15,857",55%,11.4,"Biological Sciences,Chemical Engineering,Chemi...",/world-university-rankings/imperial-college-lo...
8,University of Chicago,United States,0,University of Chicago,473,9,90,private,99.4,12,...,90.1,10,85.3,8,44 : 56,"13,525",25%,6.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-chicago
9,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,0,ETH Zurich – Swiss Federal Institute of Techno...,479,=10,100,master_account,94.3,60,...,92.0,8,76.4,21,31 : 69,"19,233",38%,14.6,"Agriculture & Forestry,Architecture,Biological...",/world-university-rankings/eth-zurich-swiss-fe...


The ratio are already there so it's pretty nice:

In [37]:
c = yolo123.copy()[['name','location','rank','stats_pc_intl_students','stats_student_staff_ratio']]
c['stats_student_staff_ratio'] = pd.to_numeric(c['stats_student_staff_ratio'])
def stringToInt(elem):
    return int(elem.replace('%',''))/100
c['stats_pc_intl_students'] = [stringToInt(x) for x in c.stats_pc_intl_students]
#sort by rank too for tie
c.sort_values(by=['stats_student_staff_ratio','rank']).head(10)

,name,location,rank,stats_pc_intl_students,stats_student_staff_ratio
105,Vanderbilt University,United States,=105,0.13,3.3
109,University of Copenhagen,Denmark,=109,0.14,4.1
11,Yale University,United States,12,0.21,4.3
12,Johns Hopkins University,United States,13,0.24,4.3
153,University of Rochester,United States,=153,0.29,4.3
97,Emory University,United States,98,0.19,4.4
16,Duke University,United States,17,0.22,4.5
114,École Polytechnique,France,115,0.36,5.1
183,Scuola Normale Superiore di Pisa,Italy,184,0.07,5.2
135,University of Zurich,Switzerland,136,0.20,5.9


In [34]:
c.sort_values(by='stats_pc_intl_students',ascending=False).head(10)
#d = c.drop(['name','rank'],axis=1).groupby('location').agg('sum')

,name,location,rank,stats_pc_intl_students,stats_student_staff_ratio
24,London School of Economics and Political Science,United Kingdom,=25,0.71,12.2
178,University of Luxembourg,Luxembourg,=179,0.57,14.6
37,École Polytechnique Fédérale de Lausanne,Switzerland,=38,0.55,11.2
7,Imperial College London,United Kingdom,8,0.55,11.4
102,Maastricht University,Netherlands,103,0.50,18.0
15,University College London,United Kingdom,16,0.49,10.5
143,University of St Andrews,United Kingdom,=143,0.48,13.0
23,Carnegie Mellon University,United States,24,0.45,13.5
120,Queen Mary University of London,United Kingdom,121,0.45,13.3
39,University of Hong Kong,Hong Kong,40,0.42,18.0


# 3. merge stuffs

In [10]:
# 2 dataframe to 1
pass





# 4. Do correlation?

In [11]:
#work
pass





# 5. Best university? (not EPFL)

In [12]:
#harward or stuff
pass


